In [67]:
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch

In [51]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__() # b, 3, 32, 32
        # 将卷积层、激活层、池化层组合在一起构成一个层结构
        layer1 = nn.Sequential()
        layer1.add_module('conv1', nn.Conv2d(3, 32, 3, 1, padding=1)) # b, 32, 32, 32
        layer1.add_module('relu1', nn.ReLU(True))
        layer1.add_module('pool1', nn.MaxPool2d(2, 2)) # b, 32, 16, 16
        self.layer1 = layer1
        
        layer2 = nn.Sequential()
        layer2.add_module('conv2', nn.Conv2d(32, 64, 3, 1, padding=1)) # b, 64, 16, 16
        layer2.add_module('relu2', nn.ReLU(True))
        layer2.add_module('pool2', nn.MaxPool2d(2, 2)) # b, 64, 8, 8
        self.layer2 = layer2
        
        layer3 = nn.Sequential() 
        layer3.add_module('conv3', nn.Conv2d(64, 128, 3, 1, padding=1)) # b, 128, 8, 8
        layer3.add_module('relu3', nn.ReLU(True))
        layer3.add_module('pool3', nn.MaxPool2d(2, 2)) # b, 128, 4, 4
        self.layer3 = layer3
        
        layer4 = nn.Sequential()
        layer4.add_module('fc1', nn.Linear(2048, 512))
        layer4.add_module('fc_relu1', nn.ReLU(True))
        layer4.add_module('fc2', nn.Linear(512, 64))
        layer4.add_module('fc_relu2', nn.ReLU(True))
        layer4.add_module('fc3', nn.Linear(64, 10))
        self.layer4 = layer4
        
    def forward(self, x):
        conv1 = self.layer1(x)
        conv2 = self.layer2(conv1)
        conv3 = self.layer3(conv2)
        fc_input = conv3.view(conv3.size(0), -1)
        fc_out = self.layer4(fc_input)
        return fc_out

In [52]:
model = SimpleCNN()
print(model)

SimpleCNN(
  (layer1): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu3): ReLU(inplace=True)
    (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (fc1): Linear(in_features=2048, out_features=512, bias=True)
    (fc_relu1): ReLU(inplace=True)
    (fc2): Linear(in_features=512, out_features=64, bias=True)
    (fc_relu2): ReLU(inplace=True)
    (fc3): Linear(in_features=64, out_features=10, bias=True)
  )
)


- 如何提取网络中指定的层结构、参数，以及如何对参数进行自定义的初始化。

In [12]:
new_model = nn.Sequential(*list(model.children())[:2])
new_model

Sequential(
  (0): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu1): ReLU(inplace=True)
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu2): ReLU(inplace=True)
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
)

In [27]:
for layer in model.children():
    print(layer)
    print('*'*30)

Sequential(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
******************************
Sequential(
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
******************************
Sequential(
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu3): ReLU(inplace=True)
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
******************************
Sequential(
  (fc1): Linear(in_features=2048, out_features=512, bias=True)
  (fc_relu1): ReLU(inplace=True)
  (fc2): Linear(in_features=512, out_features=64, bias=True)
  (fc_relu2): ReLU(inplace=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)
**********************

In [39]:
for param in model.named_parameters():
    print(param[0])

layer1.conv1.weight
layer1.conv1.bias
layer2.conv2.weight
layer2.conv2.bias
layer3.conv3.weight
layer3.conv3.bias
layer4.fc1.weight
layer4.fc1.bias
layer4.fc2.weight
layer4.fc2.bias
layer4.fc3.weight
layer4.fc3.bias


In [49]:
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        nn.init.normal(m.weight.data)
        nn.init.xavier_normal(m.weight.data)
        nn.init.kaiming_normal(m.weight.data)
    elif isinstance(m, nn.Linear):
        m.weight.data.normal_()

d:\python36\deeplearning\lib\site-packages\ipykernel_launcher.py:3: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  This is separate from the ipykernel package so we can avoid doing imports until
d:\python36\deeplearning\lib\site-packages\ipykernel_launcher.py:4: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  after removing the cwd from sys.path.
d:\python36\deeplearning\lib\site-packages\ipykernel_launcher.py:5: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  """


#### LeNet

In [54]:
class Lenet(nn.Module):
    def __init__(self):
        super(Lenet, self).__init__()
        layer1 = nn.Sequential() # b, 1, 32, 32
        layer1.add_module('conv1', nn.Conv2d(1, 6, 5, 1)) # b, 6, 28, 28
        # layer1.add_module('relu1', nn.ReLU())
        layer1.add_module('pool1', nn.MaxPool2d(2, 2)) # b, 6, 14, 14
        self.layer1 = layer1
        
        layer2 = nn.Sequential()
        layer2.add_module('conv2', nn.Conv2d(6, 16, 5, 1)) # b, 16, 10, 10
        # layer2.add_module('relu2', nn.ReLU())
        layer2.add_module('pool2', nn.MaxPool2d(2, 2)) # b, 16, 5, 5
        self.layer2 = layer2
        
        layer3 = nn.Sequential()
        layer3.add_module('fc1', nn.Linear(400, 120))
        # layer3.add_module('fc_relu1', nn.ReLU())
        layer3.add_module('fc2', nn.Linear(120, 84))
        # layer3.add_module('fc_relu2', nn.ReLU())
        layer3.add_model('fc3', nn.Linear(84, 10))
        self.layer3 = layer3
        
    def forward(self, x):
        conv1 = self.layer1(x)
        conv2 = self.layer2(conv1)
        finput = conv2.view(conv2.size(0), -1)
        out = self.layer3(finput)
        return out

#### Inception

In [55]:
import torch.nn.functional as F

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)
    
class Inception(nn.Module):
    def __init__(self, in_channels, pool_features):
        super(Inception, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 64, kernel_size=1)
        
        self.branch5x5_1 = BasicConv2d(in_channels, 48, kernel_size=1)
        self.branch5x5_2 = BasicConv2d(48, 64, kernel_size=5, padding=2)
        
        self.branch3x3db_1 = BasicConv2d(in_channels, 64, kernel_size=1)
        self.branch3x3db_2 = BasicConv2d(64, 96, kernel_size=3, padding=1)
        self.branch3x3db_3 = BasicConv2d(96, 96, kernel_size=3, padding=1)
        
        self.branch_pool = BasicConv2d(in_channels, pool_features, kernel_size=1)
        
    def forward(self, x):
        branch1x1 = self.branch1x1(x)
        
        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)
        
        branch3x3db1 = self.branch3x3db_1(x)
        branch3x3db1 = self.branch3x3db_2(branch3x3db1)
        branch3x3db1 = self.branch3x3db_3(branch3x3db1)
        
        branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
        branch_pool = self.branch_pool(branch_pool)
        
        outputs = [branch1x1, branch5x5, branch3x3db1, branch_pool]
        return torch.cat(outputs, 1) # 按深度拼接起来

#### ResNet

In [56]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            residual = self.downsample(residual)
        
        out += residual
        out = self.relu(out)
        return out

#### MNIST分类

In [70]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        layer1 = nn.Sequential() # b, 1, 32, 32
        layer1.add_module('conv1', nn.Conv2d(1, 16, kernel_size=3)) # b, 16, 26, 26
        layer1.add_module('bn1', nn.BatchNorm2d(16))
        layer1.add_module('relu1', nn.ReLU(inplace=True))
        self.layer1 = layer1
        
        layer2 = nn.Sequential()
        layer2.add_module('conv2', nn.Conv2d(16, 32, kernel_size=3)) # n, 32, 24, 24
        layer2.add_module('bn2', nn.BatchNorm2d(32))
        layer2.add_module('relu2', nn.ReLU(inplace=True))
        layer2.add_module('pool2', nn.MaxPool2d(kernel_size=2, stride=2)) # n, 32, 12, 12
        self.layer2 = layer2
        
        layer3 = nn.Sequential() 
        layer3.add_module('conv3', nn.Conv2d(32, 64, kernel_size=3)) # b, 64, 10, 10
        layer3.add_module('bn3', nn.BatchNorm2d(64))
        layer3.add_module('relu3', nn.ReLU(inplace=True)) 
        self.layer3 = layer3
        
        layer4 = nn.Sequential()
        layer4.add_module('conv4', nn.Conv2d(64, 128, kernel_size=3)) # n, 128, 8, 8
        layer4.add_module('bn4', nn.BatchNorm2d(128))
        layer4.add_module('relu4', nn.ReLU(inplace=True))
        layer4.add_module('pool4', nn.MaxPool2d(kernel_size=2, stride=2)) # n, 128, 4, 4
        self.layer4 = layer4
        
        fc = nn.Sequential()
        fc.add_module('fc1', nn.Linear(128*4*4, 1024))
        fc.add_module('fc_relu1', nn.ReLU(inplace=True))
        fc.add_module('fc2', nn.Linear(1024, 128))
        fc.add_module('fc_relu2', nn.ReLU(inplace=True))
        fc.add_module('fc3', nn.Linear(128, 10))
        self.fc = fc
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [71]:
batch_size = 64
learning_rate = 1e-2
num_epoches = 20

data_tf = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize([0.5], [0.5])]) # 减去0.5再除以0.5
train_dataset = datasets.MNIST(root='./data', train=True, transform=data_tf, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=data_tf)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model = CNN()
if torch.cuda.is_available():
    model.cuda()
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(num_epoches):
    train_loss = 0.0
    for batch_x, batch_y in train_loader:
        # shape of batch_x: [64, 1, 28, 28]
        # shape of batch_y: [64]
        if torch.cuda.is_available():
            batch_x = Variable(batch_x).cuda()
            batch_y = Variable(batch_y).cuda()
        else:
            batch_x = Variable(batch_x)
            batch_y = Variable(batch_y)
        
        out = model(batch_x)
        loss = criterion(out, batch_y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss = loss.data.item()
    print('Epoch [{}/{}], loss: {:.6f}'.format(epoch, num_epoches, train_loss))

Epoch [0/20], loss: 0.026402
Epoch [1/20], loss: 0.127695
Epoch [2/20], loss: 0.051676
Epoch [3/20], loss: 0.002557
Epoch [4/20], loss: 0.012357
Epoch [5/20], loss: 0.001415
Epoch [6/20], loss: 0.002970
Epoch [7/20], loss: 0.006304
Epoch [8/20], loss: 0.010297
Epoch [9/20], loss: 0.100375
Epoch [10/20], loss: 0.030284
Epoch [11/20], loss: 0.188129
Epoch [12/20], loss: 0.009271
Epoch [13/20], loss: 0.001434
Epoch [14/20], loss: 0.018635
Epoch [15/20], loss: 0.006563
Epoch [16/20], loss: 0.003946
Epoch [17/20], loss: 0.000297
Epoch [18/20], loss: 0.002437
Epoch [19/20], loss: 0.002046


In [50]:
16 * 5 * 5 

400